In [ ]:
import requests
import random
import json
import random
import webbrowser


In [ ]:
registryBaseUrl = "https://sandbox-sbrc.g2pconnect.org/registry/"
registryUrl = "https://sandbox-sbrc.g2pconnect.org/registry/api/v1/"
keycloakUrl = "https://sandbox-sbrc.g2pconnect.org/"
client_id='sandbox-client-1'
client_secret='f1e6404a-6829-4e9e-9fc2-d49d9e0037e4'

In [ ]:
response = requests.get('%s/health'%(registryBaseUrl))
print(response.text)
assert response.status_code == 200 

# get client token


In [ ]:
def getToken(clientId, clientSecret):
    headers = {
        'content-type': 'application/x-www-form-urlencoded',
    }

    data = {
      'client_id': clientId,
      'client_secret': clientSecret,
      'grant_type': 'client_credentials'
    }

    response = requests.post('%sauth/realms/sunbird-rc/protocol/openid-connect/token'%(keycloakUrl), headers=headers, data=data)
    print(response.json())
#     print(response.status_code)
    token = response.json()["access_token"]
    return token

token = getToken(client_id, client_secret)
print(token)

# create a schema

In [ ]:

schemaRequestBody = {
  "name": "TrainingCertificateV"+str(random.randrange(0, 100,1)),
    "schema": "{\n  \"$schema\": \"http://json-schema.org/draft-07/schema\",\n  \"type\": \"object\",\n  \"properties\": {\n    \"TrainingCertificate\": {\n      \"$ref\": \"#/definitions/TrainingCertificate\"\n    }\n  },\n  \"required\": [\n    \"TrainingCertificate\"\n  ],\n  \"title\": \"TrainingCertificate\",\n  \"definitions\": {\n    \"TrainingCertificate\": {\n      \"$id\": \"#/properties/TrainingCertificate\",\n      \"type\": \"object\",\n      \"title\": \"The TrainingCertificate Schema\",\n      \"required\": [\n        \"name\",\n        \"contact\"\n      ],\n      \"properties\": {\n        \"name\": {\n          \"type\": \"string\"\n        },\n        \"trainingTitle\": {\n          \"type\": \"string\"\n        },\n        \"contact\": {\n          \"type\": \"string\"\n        },\n        \"date\": {\n          \"type\": \"string\",\n          \"format\": \"date\"\n        },\n        \"note\": {\n          \"type\": \"string\"\n        }\n      }\n    }\n  },\n  \"_osConfig\": {\n    \"uniqueIndexFields\": [\n      \"contact\"\n    ],\n    \"ownershipAttributes\": [],\n    \"roles\": [],\n    \"inviteRoles\": [\n      \"anonymous\"\n    ],\n    \"enableLogin\": false,\n    \"credentialTemplate\": {\n      \"@context\": [\n        \"https://www.w3.org/2018/credentials/v1\",\n        \"https://gist.githubusercontent.com/dileepbapat/eb932596a70f75016411cc871113a789/raw/498e5af1d94784f114b32c1ab827f951a8a24def/skill\"\n      ],\n      \"type\": [\n        \"VerifiableCredential\"\n      ],\n      \"issuanceDate\": \"2021-08-27T10:57:57.237Z\",\n      \"credentialSubject\": {\n        \"type\": \"Person\",\n        \"name\": \"{{name}}\",\n        \"trainedOn\": \"{{trainingTitle}}\"\n      },\n      \"issuer\": \"did:web:sunbirdrc.dev/vc/skill\"\n    },\n    \"certificateTemplates\": {\n      \"html\": \"https://raw.githubusercontent.com/dileepbapat/ref-sunbirdrc-certificate/main/schemas/templates/TrainingCertificate.html\",\n      \"svg\": \"https://raw.githubusercontent.com/dileepbapat/ref-sunbirdrc-certificate/main/schemas/templates/TrainingCertificate.svg\"\n    }\n  }\n}"
}
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + token
}

response = requests.request("POST", registryUrl + "Schema", headers=headers, json=schemaRequestBody)

print(response.status_code)
print(response.text)
assert response.status_code == 200 
schemaOsid = response.json()["result"]["Schema"]["osid"]
print(schemaOsid)

In [ ]:
response

# get all schemas

In [ ]:
schemaRequestBody = {
  "entity": "Schema",
    "filters": {}
}
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + token
}

response = requests.request("POST", registryUrl + "Schema/search", headers=headers, json=schemaRequestBody)

print(response.json())
for j in response.json():
    print(j['name'])

In [ ]:
schemaRequestBody = {
  "entity": "Schema",
    "filters": {}
}
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + token
}

response = requests.request("GET", registryUrl + "Schema", headers=headers)
print(response)
print(response.json())
print(len(response.json()))

In [ ]:
schemaRequestBody = {
  "entity": "Schema",
    "filters": {}
}
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + token
}

response = requests.request("GET", registryUrl + "Schema", headers=headers)
print(response)
print(response.json())
print(len(response.json()))

# Tests for newly added schem is here:


resp = requests.get("%sapi/docs/swagger.json"%registryBaseUrl)
assert resp.status_code == 200
assert resp.json()["swagger"] == "2.0"
assert resp.json()["paths"] != None

swaggerJson = resp.json()
swaggerJson["paths"].keys()

jsonUrls = [f for f in swaggerJson["paths"].keys() if ".json" in f]
print(jsonUrls)
jsonUrl = jsonUrls[0]

resp = requests.get("%s%s"%(registryBaseUrl, jsonUrl))
assert resp.status_code == 200


In [ ]:
resp.json()
entities = list(resp.json().keys())
print("Available entities ", entities)
entity_name='TrainingCertificate'
print("Using entity %s"%entity_name)


# Create entity

In [ ]:
userId =str(random.randint(1e10,1e11))
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + token
}
resp = requests.post("%s%s"%(registryBaseUrl, '/api/v1/%s'%'TrainingCertificate'), json={
    "name":"Sunbird Learner", 
    "contact": userId, 
    "trainingTitle":"Sunbird RC Certificate Module"
    
}, headers=headers)
print(resp)
assert resp.status_code == 200 or print (resp.text)
idx = resp.json()["result"][entity_name]["osid"]


In [ ]:
print(resp.json())
resp.status_code, userId
idx = resp.json()["result"][entity_name]["osid"]

# get entity

In [ ]:
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + token,
    "Accept":"application/vc+ld+json"
}
resp = requests.get("%sapi/v1/%s/%s"%(registryBaseUrl, entity_name, idx), headers=headers)
print(json.dumps(resp.json()))
assert resp.json()["proof"]["type"] == "RsaSignature2018"


# get certificate pdf

In [ ]:
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + token,
"Accept":"application/pdf", 
    "template": "https://raw.githubusercontent.com/dileepbapat/ref-sunbirdrc-certificate/main/schemas/templates/TrainingCertificate.html"
}
resp = requests.get("%s/api/v1/%s/%s"%(registryBaseUrl, entity_name, idx), headers=headers)
(resp.status_code, resp.content)

assert resp.content[:5].decode().startswith("%PDF")
with open('sample.pdf', 'wb') as f:
    f.write(resp.content)

%system open 'sample.pdf'

# Delete schema

In [ ]:
schemaRequestBody = {
  "entity": "Schema",
    "filters": {}
}
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + token
}

response = requests.request("DELETE", registryUrl + "Schema/"+schemaOsid, headers=headers)
print(response)
print(response.json())
print(len(response.json()))

In [ ]:
# Verify QR Code

In [ ]:
webbrowser.open(keycloakUrl)

# Verify QR Code

In [80]:
webbrowser.open(keycloakUrl)

True